In [ ]:
#r "BoSSSpad.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedCavity_ConvergenceStudy");

In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedCavity_ConvergenceStudy");
// var myDb = OpenOrCreateDatabase(@"C:\Databases\HeatedCavity_ConvergenceStudy");

In [ ]:
// Find number of cells used in the finest mesh
int NCellsFinestMesh = 0;
var allsess = myDb.Sessions;
foreach(var sess in allsess){
    int c = Convert.ToInt32(sess.KeysAndQueries["Grid:NoOfCells"]);
    if( c > NCellsFinestMesh)
    NCellsFinestMesh = c;
}

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable; // The session table has to be created again in order to see the results from hconvergence

In [ ]:
int counter = 0;
string[] varNames = new string[]{"VelocityX", "VelocityY", "Pressure","Temperature"};

IEnumerable<KeyValuePair<string, double>>[] RegressionS = new IEnumerable<KeyValuePair<string, double>>[varNames.Length];

    Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 2];
    int cnt2 = -1;
    for (int iCol = 0; iCol < 2; iCol++) {
        for (int iRow = 0; iRow < 2; iRow++) {
            var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
            var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
            PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle, };
            cnt2++;
            // BoSSS data
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.PointType =  allPointTypes[cnt2 % allPointTypes.Length];
            fmt.DashType = DashTypes.Solid;            
            
            string dgDegStr = (varNames[cnt2] == "VelocityX"  || varNames[cnt2] == "VelocityY" ) ? "DGdegree:Velocity*":"DGdegree:"+varNames[cnt2];
            var Tab = BoSSSshell.WorkflowMgm.SessionTable;
            var Tab2 = Tab.ExtractColumns( dgDegStr,"Grid:hMin","Grid:NoOfCells" ,"L2Error_"+  varNames[cnt2]);
            //Extract last point, the one against we are comparing our solutions with
            Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh );
            var plot = Tab2.ToPlot("Grid:hMin","L2Error_"+  varNames[cnt2],  dgDegStr);

            int cc = 0;
            foreach(var dataGroup in plot.dataGroups){
                dataGroup.Format.PointType = myPointTypes[cc];
                cc++;
            }            
            plot.LogX = true;
            plot.LogY = true;

            plot.Title = varNames[cnt2];
            plot.TitleFont = 20;
            plot.Xlabel = "h";  
            plot.Ylabel = "L_2 Error";  
            PlotTable[iCol, iRow] = plot;

            RegressionS[cnt2] = plot.Regression();
        }
    }



In [ ]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1000,yRes:1000)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityX 
 
 
 DGdegree:Velocity*4 
 
 
 DGdegree:Velocity*4 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityY 
 
 
 DGdegree:Velocity*4 
 
 
 DGdegree:Velocity*4 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Pressure 
 
 
 DGdegree:Pressure3 
 
 
 DGdegree:Pressure3 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure2 
 
 
 DGdegree:Pressure2 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure1 
 
 
 DGdegree:Pressure1 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure0 
 
 
 DGdegree:Pressure0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Temperature 
 
 
 DGdegree:Temperature4 
 
 
 DGdegree:Temperature4 
 
 
 
 
 
 
 
 
 
 DGdegree:Temperature3 
 
 
 DGdegree:Temperature3 
 
 
 
 
 
 
 
 
 
 DGdegree:Temperature2 
 
 
 DGdegree:Temperature2 
 
 
 
 
 
 
 
 
 
 DGdegree:Temperature1 
 
 
 DGdegree:Temperature1

In [ ]:
int cnt = 0;
foreach(var reg in RegressionS){
    Console.WriteLine(varNames[cnt]);
    var regB = reg.ToArray();
    for(int j = 0; j < regB.Length; j++){       
        int dgDeg = ( regB[j].Key).Last() - '0';
        Console.Write("DG degree: "+ dgDeg+ " \t");
        double slope = regB[j].Value;
        Console.Write("Slope: " +slope + "\t");

        if((slope - (dgDeg + 1) ) >= -0.2)
        Console.WriteLine("Expected slope archieved");
        else
        Console.WriteLine("The slope value is too low");
    }
    cnt++;
}


In [ ]:
RegressionS

index,value
0,"[ [DGdegree:Velocity*4, 3.5316759819466235], [DGdegree:Velocity*3, 3.383745068415406], [DGdegree:Velocity*2, 2.872532400906743], [DGdegree:Velocity*1, 0.8788781268701168] ]"
1,"[ [DGdegree:Velocity*4, 3.599018742591269], [DGdegree:Velocity*3, 3.43603403559082], [DGdegree:Velocity*2, 2.651921284135517], [DGdegree:Velocity*1, 0.879886248506851] ]"
2,"[ [DGdegree:Pressure3, 0.7235427885694719], [DGdegree:Pressure2, 0.7265096630113661], [DGdegree:Pressure1, 0.7277479793011088], [DGdegree:Pressure0, 0.708503366199939] ]"
3,"[ [DGdegree:Temperature4, 4.6809435027077555], [DGdegree:Temperature3, 4.129419881845758], [DGdegree:Temperature2, 2.876252875851901], [DGdegree:Temperature1, 1.026511071444801] ]"
